# Recolección de datos desde GitHub

Este notebook descarga issues y comentarios del repositorio WordPress/gutenberg usando la API de GitHub.


In [1]:
import os #leer variables del sistema (github token)
import time #pausas para no saturar GitHub
import requests #conectarse a la API
import pandas as pd #armar y guardar el CSV
from dotenv import load_dotenv #leer tu archivo .env

**Cargar token + headers**
1. Lee .env

2. Busca GITHUB_TOKEN

3. Si no existe, corta con error

4. Construye headers para autenticación

In [2]:
def cargar_headers_github() -> dict: #Devuelve un diccionario
    load_dotenv() #Lee el archivo .env
    token = os.getenv("GITHUB_TOKEN")
    if not token:
        raise ValueError("No se encontró GITHUB_TOKEN en el .env")

    return {
        "Authorization": f"token {token}", # Para GitHub API - aqui github sabe que somos nosotros quienes ingresamos
        "Accept": "application/vnd.github+json", # Pide el formato JSON recomendado.
    }

headers = cargar_headers_github()
print("Token cargado")

Token cargado


**Cargar la lista de repositorios**:

* Aquí definimos que repositorios vamos a descargar y utilizar

In [3]:
repos = [
    {"owner": "AUTOMATIC1111", "repo": "stable-diffusion-webui", "slug": "auto1111_webui"},
    {"owner": "yt-dlp",       "repo": "yt-dlp",                 "slug": "ytdlp"}
]

**Funciones para descargar issues y comentarios**

Baja issues de un repo usando paginación (page) hasta llegar a max_issues. Máximo de Issues 120

In [4]:
def descargar_issues(owner, repo, estado="open", max_issues=120, por_pagina=30, pausa=0.8):
   
    base_url = f"https://api.github.com/repos/{owner}/{repo}" #Llama a la url del repo
    issues = [] #lista donde se guardarán los resultados
    pagina = 1

    while len(issues) < max_issues:
        url = f"{base_url}/issues"
        params = {"state": estado, "per_page": por_pagina, "page": pagina}
        r = requests.get(url, headers=headers, params=params)

        if r.status_code != 200:
            print("Error issues:", r.status_code, r.text[:200])
            break

        data = r.json() #Convierte la respuesta en formato Python (lista de diccionarios).
        if not data: 
            break

        for item in data:
            if "pull_request" in item: #Si el diccionario tiene la clave "pull_request", significa que es un PR, no un issue.
                continue #lo salta
            issues.append(item)
            if len(issues) >= max_issues:
                break

        pagina += 1
        time.sleep(pausa)

    return issues

**Aqui se descargan los comentarios de los repos**

Dado un issue_number, descarga sus comentarios

In [5]:
def descargar_comentarios(owner, repo, issue_number, pausa=0.25):
#Es obligatorio indicar el número del issue, porque los comentarios pertenecen a uno específico.
    base_url = f"https://api.github.com/repos/{owner}/{repo}"
    url = f"{base_url}/issues/{issue_number}/comments"
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        return []

    time.sleep(pausa) #Espera 0.25 segundos para no saturar la API
    return r.json() #Convierte la respuesta en formato JSON

**Guarda en carpetas**

Para cada repo:

1. crea carpeta ../data/{slug}

2. descarga issues

3. por cada issue, descarga comentarios

4. arma una lista de “filas” (issues y comments como registros)

5. guarda CSV

In [6]:
def asegurar_carpeta(ruta):  
    os.makedirs(ruta, exist_ok=True) #crea una carpeta.
 
for info in repos:
    owner = info["owner"]
    repo  = info["repo"]
    slug  = info["slug"]

    carpeta_repo = f"../data/{slug}" #Crea la carpeta en data
    asegurar_carpeta(carpeta_repo)

    print(f"\n=== Descargando: {owner}/{repo} ===")

    issues = descargar_issues(owner, repo, estado="open", max_issues=120) #Llama la función
    filas = [] #se guarda issue y comentario

    for issue in issues:
        num = issue["number"] #Se obtiene el número de Issues

        # Guardar issue como fila
        filas.append({ #Se crea un diccionario que representa una fila del CSV.
            "repo_slug": slug, #Ubica el repositorio
            "type": "issue", #Indica que es un issue
            "issue_number": num,  #Número de Issue
            "title": issue.get("title", "") or "", #Título
            "body": issue.get("body", "") or "", #contenido
            "created_at": issue.get("created_at", ""), #fecha
            "user": (issue.get("user") or {}).get("login", ""), #usuario
            "url": issue.get("html_url", "") #enlace
        })

        # Guardar comentarios como filas
        comentarios = descargar_comentarios(owner, repo, num) #Llama a la función
        for c in comentarios: #Cada comentario se guarda como fila
            filas.append({
                "repo_slug": slug, 
                "type": "comment", #Indica que es un comentario
                "issue_number": num,
                "title": "",
                "body": c.get("body", "") or "",
                "created_at": c.get("created_at", ""),
                "user": (c.get("user") or {}).get("login", ""),
                "url": c.get("html_url", "")
            })

    df_raw = pd.DataFrame(filas) #Aquí se convierte la lista de diccionarios en una tabla.
    print("Mensajes totales:", df_raw.shape)

    ruta_out = f"{carpeta_repo}/messages_raw.csv" #Guarda
    df_raw.to_csv(ruta_out, index=False)
    print("Guardado", ruta_out)


=== Descargando: AUTOMATIC1111/stable-diffusion-webui ===
Mensajes totales: (475, 8)
Guardado ../data/auto1111_webui/messages_raw.csv

=== Descargando: yt-dlp/yt-dlp ===
Mensajes totales: (697, 8)
Guardado ../data/ytdlp/messages_raw.csv
